In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
df = pd.read_table("train.tsv")
df.head()

Unnamed: 0   mpg  cylinders  displacement  horsepower  weight  \
0           0  39.4          4            85        70.0    2070   
1           3  18.0          6           171        97.0    2984   
2           9  32.7          6           168       132.0    2910   
3          11  38.1          4            89        60.0    1968   
4          13  28.0          4            98        80.0    2164   

   acceleration  model year  origin               car name  
0          18.6          78       3         datsun b210 gx  
1          14.5          75       1             ford pinto  
2          11.4          80       3          datsun 280-zx  
3          18.8          80       3  toyota corolla tercel  
4          15.0          72       1        dodge colt (sw)

In [3]:
y=df["mpg"].values
X=df.drop(["mpg","car name"], axis=1).values

In [21]:
from sklearn.model_selection import KFold
cv = KFold(n_splits = 3, random_state=42, shuffle = True)

In [22]:
import lightgbm as lgb
for train_index, test_index in cv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # LightGBM用のデータセットに入れる
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_test = lgb.Dataset(X_test, y_test)
    # 評価基準を設定する 
    params = {'metric' : 'rmse'}
    # 訓練データから回帰モデルを作る
    gbm = lgb.train(params, lgb_train)
    test_predicted = gbm.predict(X_test)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000732 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 206
[LightGBM] [Info] Number of data points in the train set: 132, number of used features: 8
[LightGBM] [Info] Start training from score 23.249242
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

In [23]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
# モデル評価
# rmse : 平均二乗誤差の平方根
mse = mean_squared_error(y_test, test_predicted) # MSE(平均二乗誤差)の算出
rmse = np.sqrt(mse) # RSME = √MSEの算出
print('RMSE :',rmse)

#MAE
from sklearn.metrics import mean_absolute_error
mae=mean_absolute_error(y_test, test_predicted)
print("MAE",mae)

# r2 : 決定係数
r2 = r2_score(y_test,test_predicted)
print('R2 :',r2)

RMSE : 2.539124302890439
MAE 1.9895225089874
R2 : 0.8932795449762749


In [24]:
df2 = pd.read_table("test.tsv")
X2_test=df2.drop(["car name"], axis=1)
test2_predicted = gbm.predict(X2_test)

In [25]:
outdf = pd.Series(test2_predicted, index = df2["Unnamed: 0"].values )
outdf.to_csv("submit.csv", index=True, header=False)